In [9]:
# --------------------------
# Polygon.io ETH LSTM Pipeline (Optimized)
# --------------------------

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split
import joblib
from datetime import date, timedelta
from polygon import RESTClient

# --------------------------
# USER SETTINGS
# --------------------------
API_KEY = "peeXEfM2xJR2calDdtBPMB3RW3dp7KKA"
TICKER = "X:ETHUSD"

SEQ_LEN = 150             # slightly shorter than 183 for more sequences
BATCH_SIZE = 16
EPOCHS = 50
LR = 5e-4
WEIGHT_DECAY = 1e-4
HIDDEN_SIZE = 40
NUM_LAYERS = 2
DROPOUT = 0.3
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

MODEL_SAVE_PATH = "eth_lstm_polygon.pth"
SCALER_SAVE_PATH = "eth_scaler_polygon.pkl"

# --------------------------
# Polygon.io fetch last 2 years
# --------------------------
def fetch_polygon_last_2yrs(api_key, ticker):
    client = RESTClient(api_key=api_key)
    end_date = date.today()
    start_date = end_date - timedelta(days=730)  # ~2 years
    all_data = []
    try:
        for a in client.list_aggs(
            ticker, 1, "day",
            start_date.isoformat(), end_date.isoformat(),
            limit=5000
        ):
            bar_date = (pd.to_datetime(a.timestamp, unit="ms") + timedelta(days=1)).date()
            all_data.append({
                "timestamp": bar_date,
                "open": round(a.open, 3),
                "high": round(a.high, 3),
                "low": round(a.low, 3),
                "close": round(a.close, 3),
                "volume": a.volume
            })
    except Exception as e:
        raise RuntimeError(f"Error fetching Polygon data: {e}")
    df = pd.DataFrame(all_data).sort_values("timestamp").reset_index(drop=True)
    return df

# --------------------------
# Feature engineering
# --------------------------
def add_features(df):
    df = df.copy()
    df["return"] = df["close"].pct_change()
    df["log_return"] = np.log(df["close"] / df["close"].shift(1))
    df["sma_7"] = df["close"].rolling(7).mean()
    df["sma_30"] = df["close"].rolling(30).mean()
    df["ema_14"] = df["close"].ewm(span=14).mean()
    df["volatility_7"] = df["return"].rolling(7).std()
    df["atr"] = (df["high"] - df["low"]).rolling(14).mean()
    delta = df["close"].diff()
    gain = np.where(delta>0, delta, 0)
    loss = np.where(delta<0, -delta, 0)
    avg_gain = pd.Series(gain).rolling(14).mean()
    avg_loss = pd.Series(loss).rolling(14).mean()
    rs = avg_gain / (avg_loss + 1e-9)
    df["rsi"] = 100 - (100 / (1 + rs))
    ema12 = df["close"].ewm(span=12).mean()
    ema26 = df["close"].ewm(span=26).mean()
    df["macd"] = ema12 - ema26
    df["vol_change"] = df["volume"].pct_change()
    df["obv"] = (np.sign(df["return"]) * df["volume"]).cumsum()
    df = df.replace([np.inf, -np.inf], np.nan).dropna()
    return df

# --------------------------
# Label generation
# --------------------------
def generate_next_day_labels(df, threshold=0.01):
    df = df.copy()
    df["return_next"] = df["close"].shift(-1) / df["close"] - 1
    df["label"] = 1
    df.loc[df["return_next"] > threshold, "label"] = 2
    df.loc[df["return_next"] < -threshold, "label"] = 0
    df["label"] = df["label"].fillna(1).astype(int)
    ratios = df["label"].value_counts(normalize=True).to_dict()
    return df, ratios

# --------------------------
# Sequence creation with sliding window
# --------------------------
def create_sequences(df, features, target_col="label", seq_len=SEQ_LEN, stride=1):
    sequences, labels = [], []
    data = df[features + [target_col]].values
    for i in range(0, len(data) - seq_len, stride):
        seq_x = data[i:i+seq_len, :-1]
        seq_y = data[i+seq_len, -1]
        sequences.append(seq_x)
        labels.append(seq_y)
    return np.array(sequences), np.array(labels)

# --------------------------
# Dataset class
# --------------------------
class TimeSeriesDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# --------------------------
# LSTM Classifier
# --------------------------
class LSTMClassifier(nn.Module):
    def __init__(self, input_size, hidden_size=HIDDEN_SIZE, num_layers=NUM_LAYERS,
                 num_classes=3, dropout=DROPOUT):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers,
                            batch_first=True, bidirectional=True)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_size*2, num_classes)
    def forward(self, x):
        out, _ = self.lstm(x)
        out = out[:, -1, :]
        out = self.dropout(out)
        return self.fc(out)

# --------------------------
# Class weights helper
# --------------------------
def get_class_weights(y_train, num_classes=3, device=DEVICE):
    unique_classes = np.unique(y_train).astype(int)
    weights = np.ones(num_classes, dtype=np.float32)
    computed = compute_class_weight(class_weight="balanced", classes=unique_classes, y=y_train)
    for i, cls in enumerate(unique_classes):
        weights[int(cls)] = computed[i]
    return torch.tensor(weights, dtype=torch.float32).to(device)

# --------------------------
# Training function
# --------------------------
def train_lstm_polygon(df, seq_len=SEQ_LEN, batch_size=BATCH_SIZE, epochs=EPOCHS):
    df = add_features(df)
    df, ratios = generate_next_day_labels(df)
    print("Class ratios:", ratios)

    features = [c for c in df.columns if c not in ["timestamp","label","return_next"]]

    X, y = create_sequences(df, features, seq_len=seq_len)
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, shuffle=False)

    # Scale
    n_train, t, f = X_train.shape
    n_val = X_val.shape[0]
    scaler = StandardScaler()
    X_train_2d = X_train.reshape(-1, f)
    X_val_2d = X_val.reshape(-1, f)
    X_train = scaler.fit_transform(X_train_2d).reshape(n_train, t, f)
    X_val = scaler.transform(X_val_2d).reshape(n_val, t, f)
    joblib.dump(scaler, SCALER_SAVE_PATH)

    train_loader = DataLoader(TimeSeriesDataset(X_train, y_train), batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(TimeSeriesDataset(X_val, y_val), batch_size=batch_size, shuffle=False)

    input_size = X.shape[2]
    model = LSTMClassifier(input_size)
    model.to(DEVICE)

    class_weights = get_class_weights(y_train)
    print("Class weights:", class_weights)
    criterion = nn.CrossEntropyLoss(weight=class_weights)
    optimizer = torch.optim.Adam(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)

    # Train loop
    for epoch in range(epochs):
        model.train()
        train_loss = 0
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(DEVICE), y_batch.to(DEVICE)
            optimizer.zero_grad()
            loss = criterion(model(X_batch), y_batch)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            train_loss += loss.item()
        train_loss /= len(train_loader)

        # Validation
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                X_batch, y_batch = X_batch.to(DEVICE), y_batch.to(DEVICE)
                val_loss += criterion(model(X_batch), y_batch).item()
        val_loss /= len(val_loader)
        scheduler.step(val_loss)

        if (epoch+1) % 5 == 0:
            print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

    torch.save({
        "model_state": model.state_dict(),
        "input_size": input_size,
        "features": features
    }, MODEL_SAVE_PATH)
    print(f"✅ Model saved to {MODEL_SAVE_PATH}")
    return model, features, scaler

# --------------------------
# Next-day prediction
# --------------------------
def predict_next_day(df, model, scaler, features, seq_len=SEQ_LEN):
    df = add_features(df)
    df = df[features].copy()
    df[features] = scaler.transform(df[features])
    if len(df) < seq_len:
        raise ValueError("Not enough data for prediction")
    X_latest = torch.tensor(df.values[-seq_len:], dtype=torch.float32).unsqueeze(0).to(DEVICE)
    model.eval()
    with torch.no_grad():
        probs = torch.softmax(model(X_latest), dim=1).cpu().numpy()[0]
        pred_class = int(np.argmax(probs))
    return pred_class, probs

# --------------------------
# RUN: Fetch, Train & Predict
# --------------------------
df_polygon = fetch_polygon_last_2yrs(API_KEY, TICKER)
model, features, scaler = train_lstm_polygon(df_polygon)

pred_class, probs = predict_next_day(df_polygon, model, scaler, features)
mapping = {0: "Bearish", 1: "Neutral", 2: "Bullish"}
probs_percent = [f"{p:.3f}" for p in probs]
print(f"Prediction: {mapping[pred_class]}")
print(f"Confidences -> Bearish: {probs_percent[0]}, Neutral: {probs_percent[1]}, Bullish: {probs_percent[2]}")


Class ratios: {2: 0.3594864479315264, 0: 0.3238231098430813, 1: 0.3166904422253923}
Class weights: tensor([0.9402, 1.1370, 0.9462], device='cuda:0')
Epoch 5/50, Train Loss: 1.0907, Val Loss: 1.1094
Epoch 10/50, Train Loss: 1.0757, Val Loss: 1.1140
Epoch 15/50, Train Loss: 1.0697, Val Loss: 1.1180
Epoch 20/50, Train Loss: 1.0625, Val Loss: 1.1248
Epoch 25/50, Train Loss: 1.0592, Val Loss: 1.1288
Epoch 30/50, Train Loss: 1.0514, Val Loss: 1.1304
Epoch 35/50, Train Loss: 1.0559, Val Loss: 1.1312
Epoch 40/50, Train Loss: 1.0542, Val Loss: 1.1313
Epoch 45/50, Train Loss: 1.0549, Val Loss: 1.1315
Epoch 50/50, Train Loss: 1.0502, Val Loss: 1.1317
✅ Model saved to eth_lstm_polygon.pth
Prediction: Bearish
Confidences -> Bearish: 0.398, Neutral: 0.313, Bullish: 0.289


d:\Python3.11.7\Lib\site-packages\sklearn\utils\validation.py:2742: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
